# Notes

This assignment is devoted to `pandas`. It covers indexing and filtering, and some `groupby` and `join` operations. The assignment roughly corresponds to Week 4 and the beginning of Week 5 of the course.

The main dataset you'll be using is [Titanic](https://www.kaggle.com/c/titanic). Please, note, that you must not rely on any specific location for the dataset, hence, any code like

```python
titanic_train = pd.read_csv("<location>/train.csv")
```

will fail and your notebook won't be validated and graded. Inputs to the functions are described explicitly in each case, and that's the only thing you can rely on.

In [1]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [2]:
plt.rcParams["figure.figsize"] = (6,6)

In [3]:
import numpy as np
import pandas as pd

In [4]:
STUDENT = "Gal Dahan and Evyater Shpitzer"
ASSIGNMENT = 4
TEST = False

In [5]:
# Parameters
TEST = True


In [6]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 16

# Indexing and filtering

### 1. Fixing age (1 point).

There are several known mistakes in the Titanic dataset.

Namely, [Julia Florence Siegel](https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html) (Mrs. Tyrell William Cavendish) is mistakenly marked as being 76 years old (the age she actually died, but many years after Titanic).

You must **replace the corresponding age value in the dataframe with her actual age at the time** (25) and return the dataset. Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return a copy of the input dataframe, and not perform replacement in the original dataframe. Structure and indexing must be the same as in input.

In [7]:
if not TEST:
    df1 = pd.read_csv("train.csv")
    df2 = pd.read_csv("test.csv")
    df = pd.concat([df1,df2], sort=False)
    
def fix_age(df):
    """Fix age for Julia Florence Siegel."""
    b = df.copy()
    b.loc[b["Name"].str.contains("Julia Florence Siegel"),"Age"] = 25
    return b
    

In [8]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

[Gal Dahan and Evyater Shpitzer] [Problem 1] correct!


### 2. Embarkment port distribution (1 point).

You must find the value counts for embarkment port (`Embarked` column) for the passengers, who travelled in 3-d class, were male and between 20 and 30 years old (both inclusive). No need to treat missing values separately.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return **series**, indexed with values from `Embarked`, according to `.value_counts()` method semantics:

```
S    <number of passengers in 3-d class, embarked at S, 20<=Age<=30>
C    <...>
Q    <...>
Name: Embarked, dtype: int64
```

In [3]:
def embarked_stats(df):
    """Calculate embarkment port statistics."""
    return df[(df["Age"]<=30) & (df["Age"]>=20) & (df["Pclass"]>=3) & (df["Sex"]>='male')]["Embarked"].value_counts()

In [2]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, embarked_stats)

NameError: name 'TEST' is not defined

### 3. Fill missing age values (1 point).

Some age values are missing in the Titanic dataset. You need to calculate average age over all passengers, and fill missing age values in `Age` column.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure, but without missing values in `Age` column.

In [11]:
def fix_age(df):
    """Fix missing age values."""
    new_df = df.copy()   
    new_df.fillna({'Age':new_df.Age.mean()}, inplace=True)
    
    return new_df

In [12]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

[Gal Dahan and Evyater Shpitzer] [Problem 3] failed: 
exception is 'NoneType' object has no attribute 'index'



### 4. Child travelling alone (1 point).

You must find a child (`Age<10`) on-board, who was travelling without siblings or parents and find a name of her nursemaid.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two strings, collected from `Name` column, with one being child's name and second being nursemaid's name. It's known, that there's **only one child** like this.

In [13]:
def get_nursemaid(df):
    a = list(df[df.Ticket == df[(df.Age <= 10) & (df.SibSp == 0) & (df.Parch == 0)].Ticket.iloc[0]].Name)
    return tuple((a[1], a[0]))

In [14]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_nursemaid)

[Gal Dahan and Evyater Shpitzer] [Problem 4] incorrect: 
correct = ('Emanuel, Miss. Virginia Ethel', 'Dowdell, Miss. Elizabeth')
submitted = ('Dowdell, Miss. Elizabeth', 'Emanuel, Miss. Virginia Ethel')



### 5. Port with the most children embarked (1 point).

You must find, which port had the largest percentage of children (`Age<10`) embarked, i.e. number of children divided by total number of passengers embarked.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **single string** with port letter.

In [15]:
def get_port(df):
    """Get port with the most children embarked."""
    num_of_embarked = dict(df.Embarked.value_counts())
    num_of_childrens_embarked = dict(df[df.Age < 10].Embarked.value_counts())
    max_mean = 0
    port = "Q"
    
    for k in num_of_embarked.keys():
        if max_mean < num_of_childrens_embarked[k] / num_of_embarked[k]:
            max_mean = num_of_childrens_embarked[k] / num_of_embarked[k]
            port = k
            
    return port
    

In [16]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_port)

[Gal Dahan and Evyater Shpitzer] [Problem 5] correct!


### 6. Passengers per ticket (2 points).

Calculate average and maximum number of passengers per ticket.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two values - average and mean number of passengers per ticket.

In [17]:
def get_ticket_stats(df):
    """Calculate passenger per ticket statistics."""
    g=df.groupby('Ticket') 
    b = [g.get_group(x).shape[0] for x in g.groups]
    return ( np.average(b), np.max(b))

In [18]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_ticket_stats)

[Gal Dahan and Evyater Shpitzer] [Problem 6] incorrect: 
correct = (1.4090419806243273, 11)
submitted = (11, 1.4090419806243273)



### 7. Fare per passenger (3 points).

For each individual ticket, you must calculate **fare per person for that ticket**, and then calculate averages for each class. Note, that you will need to apply `groupby` and you may consider using `.first()` of resulting `DataFrameGroupBy`. Also, caferully consider, in which order calculations are performed.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `pd.Series` with three elements, indexed by class:

```
1    <average per person fare in class 1>
2    <...>
3    <...>
Name: Pclass, dtype: float64
```

In [19]:
def get_fare_per_pass(df):
    """Calculate fare per passenger for different classes."""
    t = df[df["Pclass"] == 1].groupby("Ticket").first().reset_index()
    c = df[df["Pclass"] == 1].Ticket.value_counts().reset_index(name="count")
    c["Ticket"] = c["index"]
    y=c.merge(t)
    y['avg'] = y['Fare'] / y['count']
    
    t = df[df["Pclass"] == 2].groupby("Ticket").first().reset_index()
    c = df[df["Pclass"] == 2].Ticket.value_counts().reset_index(name="count")
    c["Ticket"] = c["index"]
    y2=c.merge(t)
    y2['avg'] = y2['Fare'] / y2['count']
    
    t = df[df["Pclass"] == 3].groupby("Ticket").first().reset_index()
    c = df[df["Pclass"] == 3].Ticket.value_counts().reset_index(name="count")
    c["Ticket"] = c["index"]
    y3=c.merge(t)
    y3['avg'] = y3['Fare'] / y3['count']
    
    f = pd.Series(data=[y.avg.mean(), y2.avg.mean(), y3.avg.mean()], index=[1,2,3],name="Pclass")
    return f 
    
    
    

In [20]:
PROBLEM_ID = 7

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_fare_per_pass)

[Gal Dahan and Evyater Shpitzer] [Problem 7] incorrect: 
correct = [Pclass
1    33.910500
2    11.411010
3     7.328701
Name: fare_per_pass, dtype: float64, Pclass
1    33.91050
2    11.41101
3     7.32959
Name: fare_per_pass, dtype: float64, Pclass
1    33.910500
2    11.411010
3     7.329146
Name: fare_per_pass, dtype: float64]
submitted = 1    31.670478
2    11.814324
3     7.700893
Name: Pclass, dtype: float64



### 8. Fill missing age values (3 points).

In problem 3 you filled missing age values with global average over all passengers. Now, you need to fill them **according to class and sex**. For example, for a female passenger from 2d class, missing age value must be filled with average age of females in 2d class.

In this problem, you may need joins and `.apply()`, although there are several ways to get the same result.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure as input, but without missing values in `Age` column.

In [21]:
def fix_age_groupped(df):
    """Fill missing age values."""
    a=[]
    female_avg = dict(df[df.Sex == 'female'].groupby('Pclass').Age.mean())
    male_avg = dict(df[df.Sex == 'male'].groupby('Pclass').Age.mean())
    
    for i  in female_avg.keys():
        a.append(df.loc[(df.Sex == 'female') & (df.Pclass == i)].fillna({'Age':female_avg[i]})) 
        
    for i  in male_avg.keys():
        a.append(df.loc[(df.Sex == 'male') & (df.Pclass == i)].fillna({'Age':male_avg[i]})) 
    
    return pd.concat(a) # this will reorder the df. if its not ok we should think in diffrent way

    
    
    

In [22]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age_groupped)

[Gal Dahan and Evyater Shpitzer] [Problem 8] correct!


### 9. Finding couples (3 points).

Based on the code from Lecture 5, build a dataframe of couples. Filter it by survival status: select those couples, in which only one of spouses survived or none of two. Built survival statistics by class, i.e. ratio of the number couples with partial survival or couples which died together, divided by total number of couples in class. If the survival status of one or both of spouses is not known, it must be considered as `0`.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `Series` with three elements indexed by values from `Pclass` column (see P7 as a reference).

In [23]:
def find_couples(df):
    """Fill missing age values."""
    new_df = df.copy()
    new_df.fillna({'Survived':0}, inplace=True) 
    
    family_names = (new_df
                     .replace(re.compile(r'\s+\(.*\)'), '')
                     .replace(re.compile("Mrs."), "Mr."))[["Name", "Sex"]]

    family_names = family_names[(family_names.Sex=="female") & family_names.Name.str.contains("Mr.")]

    family_names.reset_index().set_index("Name")#["PassengerId"]

    new_df = (new_df.join(family_names
                            .reset_index()
                            .set_index("Name")["PassengerId"],
                             on="Name", rsuffix="_Spouse"))

    new_df.rename({"PassengerId":"PassengerId_Spouse"}, axis=1, inplace=True)

    new_df = new_df.join(new_df[["Name", "Age","Survived"]],
                            on="PassengerId_Spouse", rsuffix="_Spouse")


    couple_df = new_df[new_df.PassengerId_Spouse.notnull()]
    
    couple_df["Sum_of_Survived"] = couple_df.Survived_Spouse + couple_df.Survived 
    
    res = couple_df[couple_df.Sum_of_Survived < 2].groupby("Pclass")["Sum_of_Survived"].count() / couple_df.groupby("Pclass")["Sum_of_Survived"].count()
    
    
    
    return res

In [24]:
PROBLEM_ID = 9

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_couples)

[Gal Dahan and Evyater Shpitzer] [Problem 9] correct!


<ipython-input-23-d175daa0b87b>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  couple_df["Sum_of_Survived"] = couple_df.Survived_Spouse + couple_df.Survived
<ipython-input-23-d175daa0b87b>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  couple_df["Sum_of_Survived"] = couple_df.Survived_Spouse + couple_df.Survived
<ipython-input-23-d175daa0b87b>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [25]:
if TEST:
    print(f"{STUDENT}: {int(100 * total_grade / MAX_POINTS)}")

Gal Dahan and Evyater Shpitzer: 50
